In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Notebook local path should be at `ScientificValueAgent/production_figures`.

In [ ]:
import sys

sys.path.append("..")

In [ ]:
from itertools import product
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from tqdm import tqdm
from scipy.spatial import distance_matrix
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
import warnings

In [ ]:
from sva import utils

Set some plotting defaults.

In [ ]:
utils.set_defaults()

# Two-phase sine result

In [ ]:
from sva.postprocessing import read_data, parse_results_by_acquisition_function
from sva.truth.sine2phase import (
    phase_1_sine_on_2d_raster,
    truth_sine2phase,
    sine2phase_interpolant_2d,
    sine2phase_compute_metrics_all_acquisition_functions_and_LTB,
    sine2phase_residual_2d_phase_mse,
)
from sva.truth.common import limited_time_budget, get_phase_plot_info

Get the results. This is a few GB of raw data and thus could take a little time to load.

In [ ]:
results = read_data("../results/results_23-05-02-sine2phase/")

In [ ]:
results_by_acqf = parse_results_by_acquisition_function(results)

## Core manuscript figure

In [ ]:
x, y, Z = get_phase_plot_info(phase_1_sine_on_2d_raster)
X, Y = np.meshgrid(x, y)
gradZ = np.array(np.gradient(Z))
gradZ = np.sqrt((gradZ**2).sum(axis=0))

In [ ]:
extent = (x[0], x[-1], y[0], y[-1])
scale = 1
lw = 0.5

g = np.linspace(0, 1, 200)

### Subfigure (a)

In [ ]:
color = "black"

fig, ax = plt.subplots(1, 1, figsize=(2 * scale, 2 * scale), sharex=True, sharey=True)

im = ax.imshow(Z, interpolation="bilinear", origin="lower", cmap=mpl.cm.seismic, extent=extent, alpha=0.6)

ax.text(0.05, 0.95, r"(a)", ha="left", va="top", transform=ax.transAxes, color=color)
ax.text(0.95, 0.95, "Phase 1", ha="right", va="top", transform=ax.transAxes, color=color)
ax.text(0.05, 0.05, "Phase 2", ha="left", va="bottom", transform=ax.transAxes, color=color)

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])

utils.set_grids(ax, grid=False)
ax.set_ylabel("$x_2$~[a.u.]")
ax.set_xlabel("$x_1$~[a.u.]")

# plt.savefig("figures_sine2phase/sine2phase_subfigure_a.svg", dpi=300, bbox_inches="tight")
plt.show()

### Subfigure $p(x)$ colorbar

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2 * scale, 2 * scale), sharex=True, sharey=True)

im = ax.imshow(Z, interpolation="bilinear", origin="lower", cmap=mpl.cm.seismic, extent=extent, alpha=0.6)

cbar = utils.add_colorbar(im, aspect=20)
cbar.set_ticks([0, 1])
cbar.set_label(r"$p(\mathbf{x})$", labelpad=-10)

ax.remove()

# plt.savefig("figures_sine2phase/sine2phase_p_cbar.svg", dpi=300, bbox_inches="tight")
plt.show()

### Subfigure (b)

In [ ]:
single_result = results_by_acqf["UpperConfidenceBound10"][0]
Z = single_result._record[-1]["mu"].reshape(50, 50).T
Z = Z - Z.min()
Z = Z / Z.max()
cmap = mpl.cm.magma

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2 * scale, 2 * scale), sharex=True, sharey=True)

im = ax.imshow(Z, interpolation="bilinear", origin="lower", cmap=cmap, extent=extent, alpha=0.4)

sine = 0.5 + np.sin(2.0 * np.pi * g) / 4
sine_upper = sine + 0.05
sine_lower = sine - 0.05

# ax.fill_between(g, sine_lower, sine_upper, hatch="|"*10, color=None, alpha=0, linewidth=0)
color = "k"
ax.plot(
    g,
    sine,
    f"{color}--",
    linewidth=lw,
)
# ax.plot(g, sine_upper, f"{color}--", linewidth=lw)
# ax.plot(g, sine_lower, f"{color}--", linewidth=lw)

X = single_result.data.X
ax.scatter(X[:, 0], X[:, 1], color="black", s=0.5, zorder=3)

ax.text(0.05, 0.95, r"(b)", ha="left", va="top", transform=ax.transAxes)

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])

utils.set_grids(ax, grid=False)
ax.set_ylabel("$x_2$~[a.u.]")
ax.set_xlabel("$x_1$~[a.u.]")

# plt.savefig("figures_sine2phase/sine2phase_subfigure_b.svg", dpi=300, bbox_inches="tight")
plt.show()

### Subfigure value colorbar

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2 * scale, 2 * scale), sharex=True, sharey=True)

im = ax.imshow(Z, interpolation="bilinear", origin="lower", cmap=cmap, extent=extent, alpha=0.4)

cbar = utils.add_colorbar(im, aspect=20)
cbar.set_ticks([0, 1])
cbar.set_label(r"$U(\mathbf{x})$", labelpad=-10)

ax.remove()

# plt.savefig("figures_sine2phase/sine2phase_U_cbar.svg", dpi=300, bbox_inches="tight")
plt.show()

### Subfigure (c): select metrics

In [ ]:
acquisition_function_name_maps = {
    "Linear": "LTB",
    "UpperConfidenceBound10": "UCB(10)",
    "ExpectedImprovement": "EI",
    "UpperConfidenceBound1": "UCB(1)",
    "UpperConfidenceBound20": "UCB(20)",
    "UpperConfidenceBound100": "UCB(100)",
}

In [ ]:
metrics_grid = list(range(6, 251, 6)) + [250]
metrics_grid = np.unique(metrics_grid).tolist()
metrics_grid_linear = [ii for ii in range(2, 16)]
grid_points = 200
metric = "mse"

In [ ]:
all_metrics_linear = sine2phase_compute_metrics_all_acquisition_functions_and_LTB(
    results_by_acqf,
    metrics_grid=metrics_grid,
    metrics_grid_linear=metrics_grid_linear,
    metric=metric,
    grid_points=grid_points,
    interpolation_method="linear",
    disable_pbar=False,
)

In [ ]:
only_plot = ["LTB", "EI", "UCB(10)"]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(1.5, 2))

for acquisition_function_name in acquisition_function_name_maps.keys():
    values = all_metrics_linear["metrics"][acquisition_function_name]
    metrics_grid = all_metrics_linear["metrics_grid"]
    metrics_grid_linear = all_metrics_linear["metrics_grid_linear"]
    label = acquisition_function_name_maps[acquisition_function_name]

    if only_plot is None or label in only_plot:
        mu = np.log(values).mean(axis=1)
        sd = np.log(values).std(axis=1) * 2
        label = label if label != "LTB" else "Grid"
        if label == "Grid":
            ax.plot(metrics_grid_linear, mu, label=label)
            ax.fill_between(metrics_grid_linear, mu - sd, mu + sd, linewidth=0, alpha=0.3)
        else:
            ax.plot(metrics_grid, mu, label=label)
            ax.fill_between(metrics_grid, mu - sd, mu + sd, linewidth=0, alpha=0.3)

utils.set_grids(ax)
ax.tick_params(which="minor", bottom=False, top=False)
ax.set_xticks([4, 50, 150, 250])


ax.legend(frameon=False, loc="upper right", fontsize=8)
ax.text(0.05, 0.05, r"(c)", ha="left", va="bottom", transform=ax.transAxes)
ax.set_ylim(top=0.5, bottom=-7.5)

ax.set_xlabel(r"$N$")
ax.set_ylabel(r"$\ln$(MSE)")

# plt.savefig("figures_sine2phase/sine2phase_subfigure_c.svg", dpi=300, bbox_inches="tight")
plt.show()

## Supplementary information

### Bayesian inference via clustering then classification then sampling maximal uncertainty

In [ ]:
max_queries = 250
grid_points = 100
N_exp = 10

In [ ]:
np.random.seed(123)
new_metrics = {}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    for k_clusters in [2, 3]:
        metrics_grid = list(range(3, max_queries + 1, 10))
        new_metrics[f"Bayesian_{k_clusters}"] = []

        for exp in tqdm(range(N_exp)):
            k_means = KMeans(k_clusters, n_init="auto")
            clf = LogisticRegression()
            x = list(np.random.rand(k_clusters, 2))
            y = truth_sine2phase(
                np.array(x)
            )  # This won't take single items, so we feed it the entire array each time???

            for _ in range(k_clusters + 1, max_queries + 1):
                labels = k_means.fit_predict(y)
                try:
                    clf.fit(np.array(x).reshape(-1, 2), labels)
                    x_grid, y_grid = np.meshgrid(np.linspace(0, 1, grid_points), np.linspace(0, 1, grid_points))
                    grid_array = np.column_stack((x_grid.ravel(), y_grid.ravel()))
                    proby = clf.predict_proba(grid_array)
                    shannon = np.sum(proby * np.log(1 / proby), axis=-1)
                    max_entropy_loc = grid_array[np.argmax(shannon)]
                except ValueError:  # Not enough examples of both classes to fit a logistic regression
                    max_entropy_loc = np.random.rand(2)

                x.append(max_entropy_loc)
                y = truth_sine2phase(np.array(x))

            _metrics = []
            for N in metrics_grid:
                res = sine2phase_residual_2d_phase_mse(
                    np.array(x)[:N,].reshape(-1, 2),
                    grid_points=grid_points,
                )
                _metrics.append(res)
            new_metrics[f"Bayesian_{k_clusters}"].append(_metrics)

for k_clusters in [2, 3]:
    new_metrics[f"Bayesian_{k_clusters}"] = np.array(new_metrics[f"Bayesian_{k_clusters}"])

In [ ]:
metric_grids_bayesian = []
for k_clusters in [3, 4, 5]:
    metric_grids_bayesian.append(list(range(k_clusters, max_queries + 1, 10)))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 3))

for acquisition_function_name in acquisition_function_name_maps.keys():
    values = all_metrics_linear["metrics"][acquisition_function_name]
    metrics_grid = all_metrics_linear["metrics_grid"]
    metrics_grid_linear = all_metrics_linear["metrics_grid_linear"]
    label = acquisition_function_name_maps[acquisition_function_name]

    if only_plot is None or label in only_plot:
        mu = np.log(values).mean(axis=1)
        sd = np.log(values).std(axis=1) * 2
        label = label if label != "LTB" else "Grid"
        if label == "Grid":
            ax.plot(metrics_grid_linear, mu, label=label)
            ax.fill_between(metrics_grid_linear, mu - sd, mu + sd, linewidth=0, alpha=0.3)
        else:
            ax.plot(metrics_grid, mu, label=label)
            ax.fill_between(metrics_grid, mu - sd, mu + sd, linewidth=0, alpha=0.3)

acquisition_function_name_maps_bayesian = {
    f"Bayesian_{k_clusters}": f"Bayesian_{k_clusters}_clusters" for k_clusters in [2, 3]
}
for ii, acquisition_function_name in enumerate(acquisition_function_name_maps_bayesian.keys()):
    values = new_metrics[acquisition_function_name]
    label = acquisition_function_name_maps_bayesian[acquisition_function_name]
    mu = np.nanmean(np.log(values), axis=0)
    sd = np.nanstd(np.log(values), axis=0) * 2
    ax.plot(metric_grids_bayesian[ii], mu, label=label.replace("_", " "))
    ax.fill_between(metric_grids_bayesian[ii], mu - sd, mu + sd, linewidth=0, alpha=0.3)

utils.set_grids(ax)
ax.tick_params(which="minor", bottom=False, top=False)
ax.set_xticks([4, 50, 150, 250])
# ax.set_yscale('log')

# yticks = np.array([-1, -2, -3])
# ax.set_yticks((10.0**yticks).tolist())
# ax.set_yticklabels([f"${ii}$" for ii in yticks])

# ax.axhline(0.1, color="black", linewidth=0.5, linestyle="--", zorder=-1)

# ax.legend(frameon=False, bbox_to_anchor=(1, 0.5), loc="center left")
ax.legend(frameon=False, loc="center left", fontsize=8, bbox_to_anchor=(1, 0.5))
ax.set_ylim(top=0.5, bottom=-7.5)

ax.set_xlabel(r"$N$")
ax.set_ylabel(r"$\ln$(MSE)")

# plt.savefig("figures_sine2phase/SI_sine2phase_Bayesian.pdf", dpi=300, bbox_inches="tight")
plt.show()

### Plot of the basis functions

In [ ]:
from sva.truth.common import mu_Gaussians

In [ ]:
E = np.linspace(-1, 1, 100)

In [ ]:
pure1 = mu_Gaussians(0)
pure2 = mu_Gaussians(1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 1))

ax.plot(E, pure1, color="red", label="Phase 1")
ax.plot(E, pure2, color="blue", label="Phase 2")

ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")
ax.set_xlabel("$Q$~[a.u.]")
ax.set_ylabel("$I(Q)$~[a.u.]")

# plt.savefig("figures_sine2phase/SI_sine2phase_phases.pdf", bbox_inches="tight", dpi=300)
plt.show()

### Run a single random experiment as a control

In [ ]:
from sva.experiments import Experiment, Data
from sva.value import default_asymmetric_value_function
from sva.truth.sine2phase import truth_sine2phase

In [ ]:
data = Data.from_initial_conditions(
    truth=truth_sine2phase,
    value=default_asymmetric_value_function,
    seed=123,
    how="random",
    value_kwargs={"sd": None, "multiplier": 1.0},
)

random_experiment = Experiment(
    data=data,
    acqf_signature="random"
)

In [ ]:
random_experiment.run(max_n_dat=250, pbar=True, record_gp_every=50, points_per_dimension_full_grid=50)

### 2d histograms and single-experiment plots

In [ ]:
acquisition_function_name_maps = {
    "ExpectedImprovement": "EI",
    "UpperConfidenceBound1": "UCB(1)",
    "UpperConfidenceBound10": "UCB(10)",
    "UpperConfidenceBound20": "UCB(20)",
    "UpperConfidenceBound100": "UCB(100)",
    "Random": "Random",
}

In [ ]:
all_points = dict()
for acquisition_function_name, values in results_by_acqf.items():
    tmp_metrics = [exp.data.X for exp in values]
    all_points[acquisition_function_name] = np.array(tmp_metrics)

In [ ]:
N_to_plot = [3, 50, 100, 150, 200, 249]

In [ ]:
L1 = len(acquisition_function_name_maps)
L2 = len(N_to_plot)
index = 1
cmap = mpl.cm.magma
fig, axs = plt.subplots(L1, L2, figsize=(L2, L1), sharex=True, sharey=True)

for ii, acquisition_function_name in enumerate(acquisition_function_name_maps.keys()):
    
    label = acquisition_function_name_maps[acquisition_function_name]
    
    if acquisition_function_name == "Random":
        value = random_experiment.data.X
        
        for jj, N in enumerate(N_to_plot):
            ax = axs[ii, jj]
            X = value[:N, :]  #.reshape(-1, 2)

            if N == 250:
                N = 249
            Z = random_experiment._record[N-3]["mu"].reshape(50, 50).T
            Z = Z - Z.min()
            Z = Z / Z.max()
            im = ax.imshow(Z, interpolation='bilinear', origin='lower', cmap=cmap, extent=extent, alpha=0.4)
            ax.scatter(X[:, 0], X[:, 1], s=0.1, color="black")
        
    else:
        value = all_points[acquisition_function_name]
        all_results = results_by_acqf[acquisition_function_name]
    
        for jj, N in enumerate(N_to_plot):
            ax = axs[ii, jj]
            X = value[:, :N, :]  #.reshape(-1, 2)

            if N == 250:
                N = 249
            Z = all_results[index]._record[N-3]["mu"].reshape(50, 50).T
            Z = Z - Z.min()
            Z = Z / Z.max()
            im = ax.imshow(Z, interpolation='bilinear', origin='lower', cmap=cmap, extent=extent, alpha=0.4)

            # ax.hist2d(X[:, 0], X[:, 1], bins=50, cmap="RdBu_r")
            ax.scatter(X[index, :, 0], X[index, :, 1], s=0.1, color="black")
            # ax.scatter(X[index, :3, 0], X[index, :3, 1], s=2, color="blue")
        

        

for jj, N in enumerate(N_to_plot):
    ax = axs[0, jj]
    ax.set_title(r"$N=%i$" % N)
for ii, acquisition_function_name in enumerate(acquisition_function_name_maps.keys()):
    label = acquisition_function_name_maps[acquisition_function_name]
    ax = axs[ii, -1]
    ax.text(1.05, 0.5, label, ha="left", va="center", transform=ax.transAxes, rotation=90)
for ax in axs.flatten():
    # ax.set_axis_off()
    ax.set_xticks([])
    ax.set_yticks([])

ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
ax.set_xticks([])
ax.set_yticks([])
# Use `labelpad=...` for padding when using ax.set_xlabel(...)
ax.set_ylabel("$x_2$~[a.u.]")
ax.set_xlabel("$x_1$~[a.u.]")

plt.subplots_adjust(hspace=0.03, wspace=0.03)

# plt.savefig("figures_sine2phase/SI_sine2phase_single_experiment.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
acquisition_function_name_maps = {
    "ExpectedImprovement": "EI",
    "UpperConfidenceBound1": "UCB(1)",
    "UpperConfidenceBound10": "UCB(10)",
    "UpperConfidenceBound20": "UCB(20)",
    "UpperConfidenceBound100": "UCB(100)",
}

In [ ]:
N_to_plot = [3, 50, 100, 150, 200, 250]

In [ ]:
L1 = len(acquisition_function_name_maps)
L2 = len(N_to_plot)
fig, axs = plt.subplots(L1, L2, figsize=(L2, L1), sharex=True, sharey=True)

for ii, acquisition_function_name in enumerate(acquisition_function_name_maps.keys()):
    value = all_points[acquisition_function_name]
    label = acquisition_function_name_maps[acquisition_function_name]
    for jj, N in enumerate(N_to_plot):
        ax = axs[ii, jj]
        X = value[:, :N, :].reshape(-1, 2)
        ax.hist2d(X[:, 0], X[:, 1], bins=50, cmap="RdBu_r")

for jj, N in enumerate(N_to_plot):
    ax = axs[0, jj]
    ax.set_title(r"$N=%i$" % N)
for ii, acquisition_function_name in enumerate(acquisition_function_name_maps.keys()):
    label = acquisition_function_name_maps[acquisition_function_name]
    ax = axs[ii, -1]
    ax.text(1.05, 0.5, label, ha="left", va="center", transform=ax.transAxes, rotation=90)
for ax in axs.flatten():
    ax.set_axis_off()

ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
ax.set_xticks([])
ax.set_yticks([])
# Use `labelpad=...` for padding when using ax.set_xlabel(...)
ax.set_ylabel("$x_2$~[a.u.]")
ax.set_xlabel("$x_1$~[a.u.]")

plt.subplots_adjust(hspace=0.03, wspace=0.03)

# plt.savefig("figures_sine2phase/SI_sine2phase.pdf", dpi=300, bbox_inches="tight")
plt.show()

### All acquisition functions for the metric

In [ ]:
acquisition_function_name_maps = {
    "Linear": "LTB",
    "Random": "Random",
    "ExpectedImprovement": "EI",
    "UpperConfidenceBound1": "UCB(1)",
    "UpperConfidenceBound10": "UCB(10)",
    "UpperConfidenceBound20": "UCB(20)",
    "UpperConfidenceBound100": "UCB(100)"
}

In [ ]:
only_plot = ["Random", "LTB", "EI", "UCB(1)", "UCB(10)", "UCB(20)", "UCB(100)"]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))
    
for acquisition_function_name in acquisition_function_name_maps.keys():

    values = all_metrics_linear["metrics"][acquisition_function_name]
    metrics_grid = all_metrics_linear["metrics_grid"]
    metrics_grid_linear = all_metrics_linear["metrics_grid_linear"]
    label = acquisition_function_name_maps[acquisition_function_name]

    if only_plot is None or label in only_plot:
        mu = values.mean(axis=1)
        sd = values.std(axis=1)
        if label == "LTB":
            label = "Grid"
            ax.plot(metrics_grid_linear, mu, label=label)
            ax.fill_between(metrics_grid_linear, mu - sd, mu + sd, linewidth=0, alpha=0.3)
        elif label == "Random":
            ax.plot(metrics_grid_linear[1:], mu[1:], label=label)
            ax.fill_between(metrics_grid_linear[1:], mu[1:] - sd[1:], mu[1:] + sd[1:], linewidth=0, alpha=0.3)
        else:
            ax.plot(metrics_grid, mu, label=label)
            ax.fill_between(metrics_grid, mu - sd, mu + sd, linewidth=0, alpha=0.3)

utils.set_grids(ax)
ax.tick_params(which="minor", bottom=False, top=False)
ax.set_xticks([4, 50, 150, 250])
ax.set_yscale('log')

yticks = np.array([-1, -2, -3])
ax.set_yticks((10.0**yticks).tolist())
ax.set_yticklabels([f"${ii}$" for ii in yticks])

ax.legend(frameon=False, bbox_to_anchor=(1, 0.5), loc="center left")
ax.set_ylim(top=10**-0.5)

ax.set_xlabel(r"$N$")
ax.set_ylabel(r"$\ln$(MSE)")

# plt.savefig("figures_sine2phase/SI_sine2phase_all_MSE_metrics.pdf", dpi=300, bbox_inches="tight")
plt.show()

### Length scales

In [ ]:
acquisition_function_name_maps = {
    "ExpectedImprovement": "EI",
    "UpperConfidenceBound1": "UCB(1)",
    "UpperConfidenceBound10": "UCB(10)",
    "UpperConfidenceBound20": "UCB(20)",
    "UpperConfidenceBound100": "UCB(100)"
}

In [ ]:
all_points = dict()
for acquisition_function_name, values in results_by_acqf.items():
    tmp_metrics = [exp.data.X for exp in values]
    all_points[acquisition_function_name] = np.array(tmp_metrics)

In [ ]:
n_grid = list(range(3, 251))

In [ ]:
all_length_scales = dict()
for acquisition_function_name, values in all_points.items():
    tmp_jj = []
    
    # jj is the experiment index
    for jj in tqdm(range(300)):
        
        tmp_jj_ii = []
    
        ## ii is the n-points index
        for ii in n_grid:
            p = values.squeeze()[jj, :ii, :]
            X_dist = distance_matrix(p, p)
            distance = X_dist.copy()
            distance[distance == 0.0] = np.inf
            sd = distance.min(axis=1).mean()
            tmp_jj_ii.append(sd)

        tmp_jj.append(tmp_jj_ii)

    all_length_scales[acquisition_function_name] = np.array(tmp_jj)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 1.5))

for acquisition_function_name, _ in acquisition_function_name_maps.items():
    values = all_length_scales[acquisition_function_name]
    mu = values.mean(axis=0)
    sd = values.std(axis=0)
    ax.plot(n_grid, mu, label=acquisition_function_name_maps[acquisition_function_name])
    ax.fill_between(n_grid, mu - sd, mu + sd, alpha=0.2)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylabel(r"$\langle l \rangle$")
ax.set_xlabel(r"$N$")

utils.set_grids(ax)

ax.legend(frameon=False, bbox_to_anchor=(1.05, 0.5), loc="center left")

# plt.savefig("figures_sine2phase/SI_sine2phase_sigma_with_N.pdf", bbox_inches="tight", dpi=300)
plt.show()

### What happens if you use a fixed $l$?

In [ ]:
results_by_acqf_05 = parse_results_by_acquisition_function(read_data("../results/results_23-09-02-sine2phase-sigma-0.05"))
results_by_acqf_15 = parse_results_by_acquisition_function(read_data("../results/results_23-09-02-sine2phase-sigma-0.15"))
results_by_acqf_25 = parse_results_by_acquisition_function(read_data("../results/results_23-09-02-sine2phase-sigma-0.25"))
results_by_acqf_50 = parse_results_by_acquisition_function(read_data("../results/results_23-09-02-sine2phase-sigma-0.5"))

In [ ]:
acquisition_function_name_maps = {
    "ExpectedImprovement": "EI",
    "UpperConfidenceBound1": "UCB(1)",
    "UpperConfidenceBound10": "UCB(10)",
    "UpperConfidenceBound20": "UCB(20)",
    "UpperConfidenceBound100": "UCB(100)",
}

In [ ]:
all_points_fixled_l = dict()
for name, results in zip([0.15, 0.25, 0.5], [results_by_acqf_15, results_by_acqf_25, results_by_acqf_50]):
    all_points_fixled_l[name] = dict()
    for acquisition_function_name, values in results.items():
        all_points_fixled_l[name][acquisition_function_name] = values

In [ ]:
L1 = len(acquisition_function_name_maps)
L2 = 4
index = 3
cmap = mpl.cm.magma
fig, axs = plt.subplots(L1, L2, figsize=(L2, L1), sharex=True, sharey=True)

for jj, (sigma, results) in enumerate(all_points_fixled_l.items()):

    for ii, acquisition_function_name in enumerate(acquisition_function_name_maps.keys()):
        
        experiments = results[acquisition_function_name]

        label = acquisition_function_name_maps[acquisition_function_name]
        

        ax = axs[ii, jj]
        values = np.array([exp.data.X for exp in experiments])
        X = values[index, :, :]

        Z = experiments[index]._record[-1]["mu"].reshape(50, 50).T
        Z = Z - Z.min()
        Z = Z / Z.max()
        im = ax.imshow(Z, interpolation='bilinear', origin='lower', cmap=cmap, extent=extent, alpha=0.4)

        ax.scatter(X[:, 0], X[:, 1], s=0.1, color="black")

for ii, acquisition_function_name in enumerate(acquisition_function_name_maps.keys()):
    
    experiments = results_by_acqf[acquisition_function_name]
    
    label = acquisition_function_name_maps[acquisition_function_name]

    ax = axs[ii, -1]
    values = np.array([exp.data.X for exp in experiments])
    X = values[index, :, :]

    Z = experiments[index]._record[-1]["mu"].reshape(50, 50).T
    Z = Z - Z.min()
    Z = Z / Z.max()
    im = ax.imshow(Z, interpolation='bilinear', origin='lower', cmap=cmap, extent=extent, alpha=0.4)

    ax.scatter(X[:, 0], X[:, 1], s=0.1, color="black")

        
# for jj, N in enumerate(N_to_plot):
#     ax = axs[0, jj]
#     ax.set_title(r"$N=%i$" % N)

axs[0, 0].set_title(r"$l=0.15$")
axs[0, 1].set_title(r"$l=0.25$")
axs[0, 2].set_title(r"$l=0.5$")

for ii, acquisition_function_name in enumerate(acquisition_function_name_maps.keys()):
    label = acquisition_function_name_maps[acquisition_function_name]
    ax = axs[ii, -1]
    ax.text(1.05, 0.5, label, ha="left", va="center", transform=ax.transAxes, rotation=90)
for ax in axs.flatten():
    # ax.set_axis_off()
    ax.set_xticks([])
    ax.set_yticks([])
    
plt.subplots_adjust(hspace=0.03, wspace=0.03)

ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
ax.set_xticks([])
ax.set_yticks([])
# Use `labelpad=...` for padding when using ax.set_xlabel(...)
ax.set_ylabel("$x_2$~[a.u.]")
ax.set_xlabel("$x_1$~[a.u.]")

# plt.savefig("figures_sine2phase/SI_sine2phase_fixed_l_2d_plots.pdf", dpi=300, bbox_inches="tight")
plt.show()